In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import sklearn
import xgboost as xgb
import pickle
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix, auc
from sklearn.utils import shuffle
from sklearn.svm import SVC
from itertools import product


def gini_calculate(xgbresult):    
    
    ginitable = xgbresult[['y_binary','score_100']].pivot_table(index='score_100', columns='y_binary', aggfunc=len, fill_value=0)
    ginitable['badrate'] = ginitable[0]/(ginitable[0]+ginitable[1])
    ginitable['goodrate'] = ginitable[1]/(ginitable[0]+ginitable[1])    
    btotal = sum(ginitable[0])
    gtotal = sum(ginitable[1])
    ginitable['gp'] = ginitable[1]/gtotal
    ginitable['bp'] = ginitable[0]/btotal
    ginitable['cumbp'] = ginitable['bp'].cumsum()
    ginitable['index']= ginitable['gp']*(ginitable['cumbp']+ginitable['cumbp'].shift()).fillna(0)
    return (sum(ginitable['index'])-1)

In [2]:
x_1803 = pd.read_hdf('../group1_1803.h5', key='all', mode='r')
drop = [x for x in x_1803.columns if "age" in x[:4]]
x_1803 = x_1803.drop(columns = drop)
id_1803 = pd.read_hdf('../group1_1803.h5', key='id', mode='r')
y_1803 = pd.read_hdf('../group1_1803.h5', key='y', mode='r')


x_1806 = pd.read_hdf('../group1_1806.h5', key='all', mode='r')
drop = [x for x in x_1806.columns if "age" in x[:4]]
x_1806 = x_1806.drop(columns = drop)
id_1806 = pd.read_hdf('../group1_1806.h5', key='id', mode='r')
y_1806 = pd.read_hdf('../group1_1806.h5', key='y', mode='r')

x_1809 = pd.read_hdf('../group1_1809.h5', key='all', mode='r')
drop = [x for x in x_1809.columns if "age" in x[:4]]
x_1809 = x_1809.drop(columns = drop)
id_1809 = pd.read_hdf('../group1_1809.h5', key='id', mode='r')
y_1809 = pd.read_hdf('../group1_1809.h5', key='y', mode='r')


y_1803 = y_1803.apply(lambda x : 1 if x!=0 else 0)
y_1806 = y_1806.apply(lambda x : 1 if x!=0 else 0)



In [3]:
y_1803.value_counts()

0    169508
1      2359
Name: SEC_Y_3M, dtype: int64

In [4]:
s_x_1803 = pd.read_hdf('../s1_1803.h5', key='all', mode='r')
s_id_1803 = pd.read_hdf('../s1_1803.h5', key='id', mode='r')
s_y_1803 = pd.read_hdf('../s1_1803.h5', key='y', mode='r')

s_x_1806 = pd.read_hdf('../s1_1806.h5', key='all', mode='r')
s_id_1806 = pd.read_hdf('../s1_1806.h5', key='id', mode='r')
s_y_1806 = pd.read_hdf('../s1_1806.h5', key='y', mode='r')

s_x_1809 = pd.read_hdf('../s1_1809.h5', key='all', mode='r')
s_id_1809 = pd.read_hdf('../s1_1809.h5', key='id', mode='r')
s_y_1809 = pd.read_hdf('../s1_1809.h5', key='y', mode='r')

In [5]:
x_1803['enc_customer_id'] = id_1803
x_1806['enc_customer_id'] = id_1806
x_1809['enc_customer_id'] = id_1809

s_x_1803['enc_customer_id'] = s_id_1803
s_x_1806['enc_customer_id'] = s_id_1806
s_x_1809['enc_customer_id'] = s_id_1809

In [6]:
x_1803_total = pd.merge(x_1803, s_x_1803, how="left", on='enc_customer_id')
x_1806_total = pd.merge(x_1806, s_x_1806, how="left", on='enc_customer_id')
x_1809_total = pd.merge(x_1809, s_x_1809, how="left", on='enc_customer_id')

In [16]:
x_train = pd.concat([x_1803_total, x_1806_total, x_1809_total], ignore_index=True)
y_train = pd.concat([y_1803, y_1806, s_y_1809], ignore_index=True)
x_train = x_train.drop(columns = ['enc_customer_id'])

/home/aiuser01/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  """Entry point for launching an IPython kernel.


In [19]:
y_train.shape

(524991,)

In [20]:
x_train_fold, x_test_fold, y_train_fold, y_test_fold = train_test_split(x_train, y_train, test_size = 0.2, stratify = y_train, random_state = 8522)
dtrain = xgb.DMatrix(x_train_fold, y_train_fold)
dtest = xgb.DMatrix(x_test_fold, y_test_fold)

/home/aiuser01/anaconda3/lib/python3.6/site-packages/xgboost-0.83.dev0-py3.6.egg/xgboost/core.py:591: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


In [21]:
param = {
    'objective': 'binary:logistic',
    'eval_metric':'auc',
}

In [22]:
watchlist = [(dtrain, 'train'), (dtest, 'eval')]
model = xgb.train(param, dtrain, evals = watchlist, num_boost_round=10000, early_stopping_rounds = 50)

[0]	train-auc:0.765632	eval-auc:0.752481
Multiple eval metrics have been passed: 'eval-auc' will be used for early stopping.

Will train until eval-auc hasn't improved in 50 rounds.
[1]	train-auc:0.772015	eval-auc:0.757172
[2]	train-auc:0.778013	eval-auc:0.761607
[3]	train-auc:0.784349	eval-auc:0.765017
[4]	train-auc:0.789912	eval-auc:0.766484
[5]	train-auc:0.794377	eval-auc:0.768816
[6]	train-auc:0.797926	eval-auc:0.76981
[7]	train-auc:0.801705	eval-auc:0.770753
[8]	train-auc:0.804618	eval-auc:0.770504
[9]	train-auc:0.807374	eval-auc:0.770367
[10]	train-auc:0.809846	eval-auc:0.771197
[11]	train-auc:0.812904	eval-auc:0.772999
[12]	train-auc:0.819394	eval-auc:0.777503
[13]	train-auc:0.822323	eval-auc:0.778254
[14]	train-auc:0.825951	eval-auc:0.78135
[15]	train-auc:0.829554	eval-auc:0.781905
[16]	train-auc:0.833192	eval-auc:0.781985
[17]	train-auc:0.8367	eval-auc:0.781515
[18]	train-auc:0.839756	eval-auc:0.781441
[19]	train-auc:0.842796	eval-auc:0.78098
[20]	train-auc:0.845868	eval-auc:0

In [23]:
### feature importance
s = pd.Series(model.get_score(importance_type='gain'), name='Score')
s.index.name = 'Var'
feature_importance = s.reset_index().sort_values(by='Score', ascending = False).reset_index(drop=True)

In [24]:
feature_importance.to_csv("group1_feature_importance.csv", index =False)

In [27]:
feature_importance.iloc[:50,:]

,Var,Score
0,JCIC_M_BANKCNT_MEAN_L12,240.173985
1,ever_mor,146.612023
2,ever_submor,131.884956
3,JC_224,25.977655
4,M_Loan_Amt_MEAN_L12,22.414169
5,ctbc_finmortgage_amt_max_3Y,21.913227
6,READ_PCA_11_MEAN_L3,18.326073
7,SAPBPT,17.947899
8,cnt_of_membe,17.422707
9,nums_cust_grade_max_3Y_08,17.309980


In [28]:
list(feature_importance.iloc[:50,0])

['JCIC_M_BANKCNT_MEAN_L12',
 'ever_mor',
 'ever_submor',
 'JC_224',
 'M_Loan_Amt_MEAN_L12',
 'ctbc_finmortgage_amt_max_3Y',
 'READ_PCA_11_MEAN_L3',
 'SAPBPT',
 'cnt_of_membe',
 'nums_cust_grade_max_3Y_08',
 'Read_hour4_GROW_3M',
 'TAKE_AMT_L1_DP_PERCENT',
 'READ_CLASS_23_MEAN_L3',
 'MEAN_GIVE_LOAN_AMT_L1',
 'READ_CLASS_11_L1',
 'JC_266',
 'M_Loan_Amt_ESTMEAN_L12',
 'JC_244',
 'Click_hour4_MEAN_L3',
 'TARG_Scorecard_Type_Code_TARGTERM',
 'SUM_PERM_LIMIT_GROW_12M',
 'READ_PCA_17_L0',
 'Employment_Year',
 'nums_appl_cnt_10Y',
 'MAX_Retail_Balance_GROW_3M',
 'READ_CLASS_18_GROW_3M',
 'SUM_PERM_LIMIT_NEWBETA_3M',
 'CATMcntD6M',
 'jcic_fm_bankcnt_3Y',
 'JC_199',
 'JC_085',
 'nums_cust_grade_max_3Y_Mis',
 'READ_PCA_17_MEAN_L3',
 'READ_PCA_20_L0',
 'KBamt',
 'JCIC_M_BANKCNT_ESTMEAN_L12',
 'JC_195',
 'jcic_mortgage_amt_sum_3Y',
 'sum_of_house_SPEND_PAY',
 'M_Loan_Amt_BETA_3M',
 'SUM_PERM_LIMIT_MEAN_L3',
 'SUM_OTHER_AMT_NEWBETA_12M',
 'SAPBamt',
 'READ_CLASS_17_GROW_3M',
 'READ_PCA_6_GROW_3M',
 

In [ ]:
3

In [ ]:
32